# Practice notebook for regression analysis with dependent data in NHANES

This notebook will give you the opportunity to perform some analyses
using the regression methods for dependent data that we are focusing
on in this week of the course.

Enter the code in response to each question in the boxes labeled "enter your code here".
Then enter your responses to the questions in the boxes labeled "Type
Markdown and Latex".

This notebook is based on the NHANES case study notebook for
regression with dependent data.  Most of the code that you will need
to write below is very similar to code that appears in the case study
notebook.  You will need to edit code from that notebook in small ways
to adapt it to the prompts below.

To get started, we will use the same module imports and read the data
in the same way as we did in the case study:

In [100]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import statsmodels.api as sm
import numpy as np

url = "nhanes_2015_2016.csv"
da = pd.read_csv(url)

# Drop unused columns, drop rows with any missing values.
da["DMDEDUC2"] = da.DMDEDUC2.replace({1: "lt9", 2: "x9_11", 3: "HS", 4: "SomeCollege",
                                      5: "College", 7: np.nan, 9: np.nan})
vars = ["BPXSY1", "RIDAGEYR", "RIAGENDR", "RIDRETH1", "DMDEDUC2", "BMXBMI","BPXDI1",
        "SMQ020", "SDMVSTRA", "SDMVPSU"]
da = da[vars].dropna()

# This is the grouping variable
da["group"] = 10*da.SDMVSTRA + da.SDMVPSU

## Question 1: 

Build a marginal linear model using GEE for the first measurement of diastolic blood pressure (`BPXDI1`), accounting for the grouping variable defined above.  This initial model should have no covariates, and will be used to assess the ICC of this blood pressure measure.

In [101]:
# enter your code here
model = sm.GEE.from_formula("BPXDI1 ~ 1",groups = "group",cov_struct = sm.cov_struct.Exchangeable(),data = da)
result = model.fit()
result.cov_struct.summary()


'The correlation between two observations in the same cluster is 0.031'

In [102]:
#icc for systolic blood pressure
model = sm.GEE.from_formula("BPXSY1 ~ 1",groups = "group",cov_struct = sm.cov_struct.Exchangeable(),data = da)
result = model.fit()
result.cov_struct.summary()

'The correlation between two observations in the same cluster is 0.030'

__Q1a.__ What is the ICC for diastolic blood pressure?  What can you
  conclude by comparing it to the ICC for systolic blood pressure?

It is 0.031.ICC for diastolic blood pressure is less than systolic blood pressure

## Question 2: 

Take your model from question 1, and add gender, age, and BMI to it as covariates.

In [103]:
da["RIAGENDRx"] = da.RIAGENDR.replace({1:"Male",2:"Female"})

In [104]:
# enter your code here
model = sm.GEE.from_formula("BPXDI1 ~ RIAGENDRx + RIDAGEYR + BMXBMI",cov_struct = sm.cov_struct.Exchangeable(),groups = "group",
                           data = da)
result = model.fit()
result.cov_struct.summary()

'The correlation between two observations in the same cluster is 0.030'

__Q2a.__ What is the ICC for this model?  What can you conclude by comparing it to the ICC for the model that you fit in question 1?

It is 0.30.The previous model ICC has slightly higher intra-class correlation which means that observations inside clusters has higher correlation in the previous model than this new model

## Question 3: 

Split the data into separate datasets for females and for males and fit two separate marginal linear models for diastolic blood pressure, one only for females, and one only for males.

In [105]:
#Creating datasets
#for males
da_males = da[da["RIAGENDRx"]=="Male"]
#for females
da_females = da[da["RIAGENDRx"]=="Female"]


In [106]:
#model for males
model1 = sm.GEE.from_formula("BPXDI1 ~ 1",groups = "group",
                            cov_struct = sm.cov_struct.Exchangeable(),data = da_males)
result_m = model1.fit()
result_m.cov_struct.summary()

'The correlation between two observations in the same cluster is 0.035'

In [107]:
#model for females
model2 = sm.GEE.from_formula("BPXDI1 ~ 1",groups = "group",
                            cov_struct = sm.cov_struct.Exchangeable(),data = da_females)
result_fm = model2.fit()
result_fm.cov_struct.summary()

'The correlation between two observations in the same cluster is 0.029'

In [96]:
# table = pd.DataFrame({"male_params":result_m.params,"males_se":result_m.bse,
#                      "female_params":result_fm.params,"females_se":result_fm.bse})
# table

__Q3a.__ What do you learn by comparing these two fitted models?

male model has higher intra class correlation than female model.

## Question 4: 

Using the entire data set, fit a multilevel model for diastolic blood pressure, predicted by age, gender, BMI, and educational attainment.  Include a random intercept for groups.

In [97]:
# THIS CODE TAKES A LOT OF TIME IN EXECUTION,BEWARE!!!

model = sm.MixedLM.from_formula("BPXDI1 ~ RIDAGEYR + RIAGENDRx + BMXBMI + C(DMDEDUC2)",
                                groups="group", data=da)
result = model.fit()
result.summary()


<class 'statsmodels.iolib.summary2.Summary'>
"""
                Mixed Linear Model Regression Results
=====================================================================
Model:                MixedLM     Dependent Variable:     BPXDI1     
No. Observations:     5100        Method:                 REML       
No. Groups:           30          Scale:                  154.2352   
Min. group size:      105         Likelihood:             -20111.9561
Max. group size:      226         Converged:              Yes        
Mean group size:      170.0                                          
---------------------------------------------------------------------
                           Coef.  Std.Err.   z    P>|z| [0.025 0.975]
---------------------------------------------------------------------
Intercept                  65.646    1.032 63.631 0.000 63.624 67.668
RIAGENDRx[T.Male]           2.755    0.351  7.855 0.000  2.068  3.443
C(DMDEDUC2)[T.HS]          -1.093    0.521 -2.098 0.036 -2.115 -0.072
C(DMDEDUC2)[T.SomeCollege] -0.428    0.484 -0.883 0.377 -1.377  0.521
C(DMDEDUC2)[T.lt9]         -0.955    0.641 -1.488 0.137 -2.212  0.303
C(DMDEDUC2)[T.x9_11]       -0.219    0.630 -0.347 0.729 -1.454  1.016
RIDAGEYR                   -0.039    0.010 -3.871 0.000 -0.059 -0.019
BMXBMI                      0.186    0.026  7.283 0.000  0.136  0.236
group Var                   4.170    0.108                           
=====================================================================

"""

__Q4a.__ How would you describe the strength of the clustering in this analysis?

In [109]:
ICC = 4.170 / 154.24                   #Scale is 154.24 from above table
ICC 

0.027035788381742736

Icc is small but not negligible so there is significat intra class correlation  

In [108]:
# THIS CODE TAKES A LOT OF TIME IN EXECUTION,BEWARE!!!
model = sm.MixedLM.from_formula("BPXDI1 ~ RIDAGEYR + RIAGENDRx + BMXBMI + C(DMDEDUC2)",groups = "group",
                               vc_formula = {"RIDAGEYR":"1 + RIDAGEYR"},data = da)
result = model.fit()
result.summary()
#THE vc_formula function adds the intercept to the age

/opt/conda/lib/python3.6/site-packages/statsmodels/regression/mixed_linear_model.py:2045: ConvergenceWarning: The MLE may be on the boundary of the parameter space.
  warnings.warn(msg, ConvergenceWarning)


<class 'statsmodels.iolib.summary2.Summary'>
"""
                Mixed Linear Model Regression Results
=====================================================================
Model:                MixedLM     Dependent Variable:     BPXDI1     
No. Observations:     5100        Method:                 REML       
No. Groups:           30          Scale:                  154.2636   
Min. group size:      105         Likelihood:             -20112.4351
Max. group size:      226         Converged:              Yes        
Mean group size:      170.0                                          
---------------------------------------------------------------------
                           Coef.  Std.Err.   z    P>|z| [0.025 0.975]
---------------------------------------------------------------------
Intercept                  65.444    0.962 68.054 0.000 63.559 67.328
RIAGENDRx[T.Male]           2.729    0.351  7.780 0.000  2.041  3.416
C(DMDEDUC2)[T.HS]          -1.091    0.520 -2.100 0.036 -2.110 -0.073
C(DMDEDUC2)[T.SomeCollege] -0.445    0.483 -0.922 0.357 -1.391  0.501
C(DMDEDUC2)[T.lt9]         -0.995    0.642 -1.551 0.121 -2.254  0.263
C(DMDEDUC2)[T.x9_11]       -0.224    0.628 -0.357 0.721 -1.455  1.006
RIDAGEYR                   -0.034    0.012 -2.759 0.006 -0.059 -0.010
BMXBMI                      0.188    0.026  7.363 0.000  0.138  0.238
RIDAGEYR Var                0.002    0.000                           
=====================================================================

"""

__Q4b:__ Include a random intercept for age, and describe your findings.

Estimated variance of random intercept for age is 0.002.
So standard deviation is 0.045
now,(-0.034 - 0.045,-0.034 + 0.045) is the range of values for average diastolic blood pressure.